In [ ]:
import os
import re

input_path = 'shanhaijing.txt'  # 源文本放在当前工作目录
out_dir = 'shanhaijing'
os.makedirs(out_dir, exist_ok=True)

def safe_name(name: str) -> str:
    # 去掉开头的*并清理成安全文件名
    name = name.strip().lstrip('*').strip()
    return re.sub(r'[\\/:*?"<>|]+', '_', name) or 'chapter'

def unique_txt_path(dir_, base):
    path = os.path.join(dir_, f'{base}.txt')
    i = 1
    while os.path.exists(path):
        i += 1
        path = os.path.join(dir_, f'{base}_{i}.txt')
    return path

chap_title = None
chap_lines = []

with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.lstrip().startswith('*'):
            # 写出上一章
            if chap_title is not None:
                base = safe_name(chap_title)
                out_path = unique_txt_path(out_dir, base)
                with open(out_path, 'w', encoding='utf-8') as w:
                    w.write(chap_title.strip() + '\n')
                    w.writelines(chap_lines)
            # 开始新章
            chap_title = line.strip()
            chap_lines = []
        else:
            chap_lines.append(line)

# 写出最后一章
if chap_title is not None:
    base = safe_name(chap_title)
    out_path = unique_txt_path(out_dir, base)
    with open(out_path, 'w', encoding='utf-8') as w:
        w.write(chap_title.strip() + '\n')
        w.writelines(chap_lines)

print('完成：已按“*”分章并保存到 shanhaijing 文件夹')

In [2]:
import os, glob
import pandas as pd

folder = 'csv_by_category'
out_file = 'merged.csv'

# 递归查找所有子文件夹里的 CSV
pattern = os.path.join(folder, '**', '*.csv')
files = sorted(glob.glob(pattern, recursive=True))

if not files:
    print('无CSV文件')
else:
    dfs = [pd.read_csv(f) for f in files]
    merged = pd.concat(dfs, ignore_index=True)
    merged.to_csv(out_file, index=False, encoding='utf-8')
    print(f'合并完成：{out_file}，来源 {len(files)} 个文件，合计 {len(merged)} 行数据（仅一行表头）')

合并完成：merged.csv，来源 46 个文件，合计 2709 行数据（仅一行表头）


In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    # 如果没有配置环境变量，请用API Key将下行替换为：api_key="sk-xxx"
    # 新加坡和北京地域的API Key不同。获取API Key：https://help.aliyun.com/zh/model-studio/get-api-key
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    # 以下是北京地域base_url，如果使用新加坡地域的模型，需要将base_url替换为：https://dashscope-intl.aliyuncs.com/compatible-mode/v1
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
name='天虞'
category='人名'
description='有人反臂'

completion = client.chat.completions.create(
    model="qwen-max",
    messages=[
        {
            "role": "system",
            "content": "你是《山海经》研究专家和AI图像生成提示词专家。请基于形象,分类和描述生成适合AI绘画的prompt，如果有描述，详细扩写描述部分内容，不要有其他内容，如果没有则自由发挥，利用你的知识库搜索相关的神话学、考古学资料，参考古代插画风格和传统中国美学的彩色水墨风格,只返回prompt，以prompt开头。"
        },
        {
            "role": "user",
            "content": f"名字：{name}，分类：{category}，描述：{description}", 
        },
    ],
    extra_body={"enable_search": True}
)

json_string = completion.choices[0].message.content
print(json_string)